<a href="https://colab.research.google.com/github/johnpcampbell/SourceConScraper/blob/master/Accuranker_Ad_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests




SyntaxError: invalid syntax (<ipython-input-1-526777a74bea>, line 1)

In [2]:
# If 'tqdm' isn't installed, install it via pip.
!pip install tqdm
!pip install beautifulsoup4


In [ ]:
1c49ec6860690b07e66ddf0a123ce8cd15f4ca78

In [3]:
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import csv
import os
from tqdm import tqdm

# Constants
API_TOKEN = '1c49ec6860690b07e66ddf0a123ce8cd15f4ca78'  # Replace this with your actual API token
BASE_URL = 'https://app.accuranker.com/api/v4'
HEADERS = {
    'Authorization': f'Token {API_TOKEN}',
    'Content-Type': 'application/json'
}

timestampcsv = datetime.now().strftime('%Y-%m-%d-%H-%M')
# Function to create a directory for HTML files with the current timestamp
def create_directory():
    timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M')
    csv_filename = f'ads_data_{timestamp}.csv'
    directory_name = f'html-files-{timestamp}'
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
    return directory_name

# Function to fetch domains and return a JSON object or None
def fetch_domains():
    response = requests.get(f'{BASE_URL}/domains/?fields=domain,display_name,id', headers=HEADERS)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch domains. Status code: {response.status_code}, Message: {response.text}")
        return []

# Function to get keywords
def get_keywords(domain_id):
    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    url = (f'{BASE_URL}/domains/{domain_id}/keywords/?filter=6548&'
           f'period_from={yesterday}&period_to={yesterday}&'
           'fields=id,keyword,tags,search_type,ranks.id,ranks.created_at,'
           'ranks.rank,ranks.highest_ranking_page,ranks.page_serp_features.ads_top,'
           'ranks.page_serp_features.ads_bottom')
    response = requests.get(url, headers=HEADERS)
    return response.json() if response.status_code == 200 else []

# Function to extract ad info from HTML
def extract_ad_info(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    ads_data = []
    for div in soup.find_all('div', class_='uEierd'):
        ad_data = {
            'url': '',
            'headline': '',
            'description': ''
        }
        a_tag = div.find('a', class_='sVXRqc')
        if a_tag and 'href' in a_tag.attrs:
            ad_data['url'] = a_tag['href']

        headline_div = div.find('div', class_='CCgQ5')
        if headline_div:
            ad_data['headline'] = headline_div.get_text(strip=True)

        # Find the div with the class 'Va3FIb r025kc lVm3ye' and extract its text
        description_div = div.find('div', class_='Va3FIb r025kc lVm3ye')
        if description_div:
            # Remove <em> tags from the description
            for em_tag in description_div.find_all('em'):
                em_tag.decompose()
            ad_data['description'] = description_div.get_text(strip=True)

        ads_data.append(ad_data)
    return ads_data


# Create the directory for HTML files
html_directory = create_directory()

# Fetch domains, keywords, extract ad info, and write to CSV file as before
domains = fetch_domains()
if domains:
    print("Please select a domain to process:")
    for index, domain in enumerate(domains):
        print(f"{index + 1}: {domain['display_name']} (ID: {domain['id']})")

    selected_index = int(input("Enter the number of the domain you want to use: ")) - 1
    selected_domain = domains[selected_index]
    domain_id = selected_domain['id']
    domain_name = selected_domain['display_name']

    keywords = get_keywords(domain_id)

    csv_data = []
    for keyword in tqdm(keywords, desc="Processing keywords", unit="keyword"):
        keyword_id = keyword['id']
        keyword_text = keyword['keyword']

        for rank in tqdm(keyword['ranks'], desc="Processing ranks", leave=False, unit="rank"):
            rank_id = rank['id']
            created_at = rank['created_at']
            rank_value = rank['rank']
            rank_html_response = requests.get(f'{BASE_URL}/domains/{domain_id}/keywords/{keyword_id}/ranks/{rank_id}/', headers=HEADERS)
            if rank_html_response.status_code == 200:
                rank_html_data = rank_html_response.json()
                html_content = rank_html_data.get('html', '')

                # Save the HTML content to a file
                html_filename = f"{html_directory}/{rank_id}.html"
                with open(html_filename, 'w', encoding='utf-8') as html_file:
                    html_file.write(html_content)
                tqdm.write(f"Saved HTML file: {html_filename}")

                # Extract ads data from the HTML
                ads_data = extract_ad_info(html_content)
                for ad_index, ad in enumerate(ads_data):
                    csv_row = {
                        'domain_id': domain_id,
                        'domain_name': domain_name,
                        'keyword_id': keyword_id,
                        'keyword': keyword_text,
                        'rank_id': rank_id,
                        'created_at': created_at,  # new field
                        'rank': rank_value,       # new field
                        'ad': ad_index + 1,
                        'url': ad['url'],
                        'headline': ad['headline'],
                        'description': ad['description']
                    }
                    csv_data.append(csv_row)
            else:
                tqdm.write(f"Failed to retrieve HTML for rank ID {rank_id}")

    # Write to CSV
csv_file = f'ads_data_{timestampcsv}.csv'
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    fieldnames = ['domain_id', 'domain_name', 'keyword_id', 'keyword', 'rank_id', 'created_at', 'rank', 'ad', 'url', 'headline', 'description']
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for row in csv_data:
        writer.writerow(row)
print(f"CSV file created successfully: {csv_file}")



Please select a domain to process:
1: AJBell (ID: 498605)
2: AJBell (AJBell tracking using youinvest.co.uk) (ID: 496400)
3: audible.co.uk (Audible UK (2021)) (ID: 496523)
4: AVEVA Australia (ID: 504567)
5: AVEVA Germany (ID: 504783)
6: AVEVA HMI PPC Tracking (ID: 496314)
7: AVEVA India (ID: 504761)
8: Aveva Integration (ID: 496380)
9: AVEVA (/pt-br/) (ID: 504758)
10: AVEVA UK (ID: 504763)
11: AVEVA US (ID: 496493)
12: AVEVA USA (Test) (ID: 496356)
13: delta.lcp.com/ (ID: 502158)
14:  (ID: 496285)
15: experian.co.uk (11692) (ID: 496461)
16: Experian (Reddit Test) (ID: 506795)
17: FOREX Trading LSEG (FOREX Trading LSEG) (ID: 496315)
18: Forex Trading Refinitiv (FOREX Trading Refinitiv) (ID: 496344)
19: Freesat (live) (ID: 496345)
20: HI UK - Home Care in Locations (ID: 514622)
21: Home Instead IE (ID: 503562)
22: Home Instead IE Careers (ID: 515134)
23: Home Instead Local (ID: 496229)
24: Home Instead UK (ID: 514300)
25: https://www.dominusrealestate.co.uk/ (ID: 496374)
26: https://www.l

Processing keywords:   0%|          | 0/1009 [00:00<?, ?keyword/s]

Processing ranks:   0%|          | 0/1 [00:01<?, ?rank/s]

Saved HTML file: html-files-2024-04-26-10-22/14516335153.html



Processing keywords:   0%|          | 1/1009 [00:01<29:46,  1.77s/keyword]

Processing ranks:   0%|          | 0/1 [00:01<?, ?rank/s]

Saved HTML file: html-files-2024-04-26-10-22/14516335857.html


...

Saved HTML file: html-files-2024-04-26-10-22/14516414292.html



Processing keywords: 100%|█████████▉| 1008/1009 [30:06<00:01,  1.96s/keyword]

Processing ranks:   0%|          | 0/1 [00:01<?, ?rank/s]

Saved HTML file: html-files-2024-04-26-10-22/14516417217.html



Processing keywords: 100%|██████████| 1009/1009 [30:08<00:00,  1.79s/keyword]

CSV file created successfully: ads_data_2024-04-26-10-22.csv
